<a href="https://colab.research.google.com/github/iIsunnyIi/TransformerDemo/blob/main/PyTorch_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Predicting the next word using GPT-**2**

In [1]:
!pip install pytorch-transformers

     |████████████████████████████████| 184kB 11.4MB/s 
     |████████████████████████████████| 133kB 31.7MB/s 
     |████████████████████████████████| 1.1MB 32.7MB/s 
     |████████████████████████████████| 890kB 51.1MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 7.1MB 36.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=fe2e98b2409139a2d31ea635d530638269582285af57e92609780b1f3f2f02de
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.19.37 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [2]:
# Import required libraries
import torch
from pytorch_transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Encode a text inputs
text = "What is the fastest car in the"
indexed_tokens = tokenizer.encode(text)

# Convert indexed tokens in a PyTorch tensor
tokens_tensor = torch.tensor([indexed_tokens])

# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set the model in evaluation mode to deactivate the DropOut modules
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

# Get the predicted next sub-word
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])

# Print the predicted word
print(predicted_text)

100%|██████████| 548118077/548118077 [00:10<00:00, 53297919.75B/s]


 What is the fastest car in the world


##Natural Language Generation using GPT-2, Transformer-XL and XLNet

In [3]:
!git clone https://github.com/huggingface/pytorch-transformers.git

Cloning into 'pytorch-transformers'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 56836 (delta 0), reused 5 (delta 0), pack-reused 56829
Receiving objects: 100% (56836/56836), 42.29 MiB | 27.25 MiB/s, done.
Resolving deltas: 100% (39856/39856), done.


In [4]:
!python pytorch-transformers/examples/run_generation.py \
    --model_type=gpt2 \
    --length=100 \
    --model_name_or_path=gpt2 \

python3: can't open file 'pytorch-transformers/examples/run_generation.py': [Errno 2] No such file or directory


In [5]:
!python pytorch-transformers/examples/run_generation.py \
    --model_type=xlnet \
    --length=50 \
    --model_name_or_path=xlnet-base-cased \

python3: can't open file 'pytorch-transformers/examples/run_generation.py': [Errno 2] No such file or directory


In [6]:
!python pytorch-transformers/examples/run_generation.py \
    --model_type=transfo-xl \
    --length=100 \
    --model_name_or_path=transfo-xl-wt103 \

python3: can't open file 'pytorch-transformers/examples/run_generation.py': [Errno 2] No such file or directory


##Training a Masked Language Model for BERT

In [ ]:
So why are we doing this? The model learns the rules of the language during the training process. And we’ll soon see how effective this process is.

First, let’s prepare a tokenized input from a text string using BertTokenizer:

In [7]:
import torch
from pytorch_transformers import BertTokenizer, BertModel, BertForMaskedLM

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)

100%|██████████| 231508/231508 [00:00<00:00, 18047262.85B/s]


The next step would be to convert this into a sequence of integers and create PyTorch tensors of them so that we can use them directly for computation:

In [8]:
# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 8
tokenized_text[masked_index] = '[MASK]'
assert tokenized_text == ['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer', '[SEP]']

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

Notice that we have set [MASK] at the 8th index in the sentence which is the word ‘Hensen’. This is what our model will try to predict.

Now that our data is rightly pre-processed for BERT, we will create a Masked Language Model. Let’s now use BertForMaskedLM to predict a masked token:

In [9]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
    outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    predictions = outputs[0]

# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
assert predicted_token == 'henson'
print('Predicted token is:',predicted_token)

100%|██████████| 440473133/440473133 [00:05<00:00, 78653312.70B/s]


Predicted token is: henson
